In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pydicom
import os
import os.path
import pyvista as pv
from scipy.stats import *

# Utilità di display dell'immagine
def plot_images(img_stack, *, labels=None, figdim=5, colormap=plt.cm.gray, **imshowkwarg):
    """
    Funzione di utilità per mostrare una grigli di immagini, impostando la lista delle legende
    e, opzionalmente, la colormap che di default è impostata a livelli di grigio
    """    
    nimag = len(img_stack)

    # creiamo il layout
    nrows = 1 if nimag <= 3 else 2 if nimag <= 6 else 3 if nimag <=12 else 4 if nimag <= 16 else 5
    ncols = nimag if nrows == 1 else 2 if nimag == 4 else 3 if nimag <=9 else 4 if nimag <= 16 else 5

    # generiamo la figura con un canvas ce riserva 5x5 pollici a immagine
    # e i riferimenti alle singole immagini
    fig, axs = plt.subplots(nrows,ncols,squeeze=False,figsize=(figdim*ncols, figdim*nrows))
    
    img = 0 # contatore delle immagini

    for r in range(nrows):
        for c in range(ncols):
            # eliminiamo lo stile del grafico
            axs[r, c].tick_params(axis='both',\
                    which='both',\
                    bottom=False,top=False,right=False,left=False,\
                    labelbottom=False,labelleft=False)
            plt.gca().spines['top'].set_visible(False)
            plt.gca().spines['bottom'].set_visible(False)
            plt.gca().spines['left'].set_visible(False)
            plt.gca().spines['right'].set_visible(False)

            # se ci troviamo nella posizione di una
            # immagine allora la mostriamo
            if r*ncols + c < nimag:
                if labels != None:
                    axs[r, c].set_title(labels[img])
            
                axs[r, c].imshow(img_stack[img], cmap=colormap, **imshowkwarg)

                img += 1 # immagine successiva
    
    return axs

# Utilità di caricamento del volume DICOM
def load_dcm_volume(directory):
    """
    Funzione che carica un volume di dati DICOM da una cartella
    assumendo che questa contenga solamente un insieme di file .dcm
    """

    # leggiamo la lista dei file nella cartella
    files = os.listdir(directory)
    files.sort()
    
    # leggiamo il primo file per ottenere le dimensioni e il tipo della singola slice
    file = pydicom.dcmread(f'{directory}/{files[0]}')
    
    # creiamo il volume
    volume = np.full((len(files),file.pixel_array.shape[0],file.pixel_array.shape[1]),\
                    1,dtype=file.pixel_array.dtype)

    # carichiamo la prima slice, già letta, e successivamente le altre 
    volume[0,:,:] = file.pixel_array

    for i in range(1,len(files)):
        volume[i,:,:] = pydicom.dcmread(f'{directory}/{files[i]}').pixel_array
    
    vol = np.copy(volume)

    if file.SeriesDescription != '' and file.SeriesDescription.find('Sag') != -1:
        vol=np.reshape(vol,(volume.shape[1],volume.shape[2],volume.shape[0]))

        for i in range(volume[:,0,:].shape[1]):
            vol[i,:,:] = volume[:,i,:].T
    
    if file.SeriesDescription != '' and file.SeriesDescription.find('Cor') != -1:
        vol=np.reshape(vol,(volume.shape[1],volume.shape[0],volume.shape[2]))

        for i in range(volume[:,0,:].shape[0]):
            vol[i,:,:] = volume[:,i,:]
    
    return vol


In [2]:
dcm_volume = load_dcm_volume('Data/series-00000')


In [3]:
def opacity(low, high):
    
    opacity_vector = np.zeros((255,),dtype=float)
    
    step = high - low // 3
    
    for i in range (low, high):
        opacity_vector[i]= opacity_vector[i-1] + 1/step if i < low + step \
                            else 1 if i < low + 2*step \
                            else opacity_vector[i-1] - 1/step
    
    return opacity_vector

In [ ]:

print(describe(dcm_volume,axis=None), median_abs_deviation(dcm_volume,axis=None))

opacity = opacity(200, 240)

p = pv.Plotter()
p.add_volume(dcm_volume, cmap="viridis", opacity=opacity, blending='composite', shade=True)

p.show()

In [ ]:
from pyvista import examples

vol = examples.download_knee_full()

vol.plot(volume=True, cmap="bone", cpos=cpos)

In [ ]:
p = pv.Plotter()
p.add_volume(vol, cmap="bone", opacity="sigmoid")
p.show()